In [1]:
!pip install numpy
!pip install scipy
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 860.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 1.4 MB/s eta 0:00:0000:0100:01


https://towardsdatascience.com/similarity-search-knn-inverted-file-index-7cab80cc0e79

In [2]:
import numpy as np
from scipy.cluster.vq import vq, kmeans2
from scipy.spatial.distance import cdist
import faiss

# Load Data

In [4]:
data = np.load('features/all_feartures_trt.npz')

In [5]:
feature = data['feature']
path = data['path']

In [6]:
d = feature.shape[1]
nlist = 100
M = 16
nbits = 8

In [8]:
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, M, nbits)

In [9]:
index.train(feature)
index.add(feature)

In [10]:
index.nprobe = 100

In [11]:
D, I = index.search(feature[:1], 2)

In [12]:
I[0]

array([   0, 2390])

In [13]:
d = 10                           # dimension
nb = 256                      # database size
nq = 1                       # nb of queries
np.random.seed(1234)             # make reproducible

# Use numpy.tile to repeat values along rows
xb = np.tile(np.arange(1, nb + 1), (d, 1)).astype('float32').T
print(xb.shape)
xq = xb[5,:]
xq = xq.reshape(1,xq.shape[0])
print(xq)
print(xq.shape)

(256, 10)
[[6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]]
(1, 10)


In [14]:
import numpy as np

d = 10                           # dimension
nb = 256                      # database size
nq = 1                       # nb of queries
np.random.seed(1234)             # make reproducible

# Use numpy.tile to repeat values along rows
xb = np.tile(np.arange(1, nb + 1), (d, 1)).astype('float32').T
print(xb.shape)
xq = xb[5,:]
xq = xq.reshape(1,xq.shape[0])
print(xq)
print(xq.shape)





nlist = 100
m = 5 # number of codebook
k = 10 
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                  # 8 specifies that each sub-vector is encoded as 8 bits
index.train(xb)
index.add(xb)
D, I = index.search(xb[:1], k) # sanity check
print(I)
print(I.shape)
index.nprobe = 10            # make comparable with experiment above
D, I = index.search(xq, k)# search

print(I[-1:])

(256, 10)
[[6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]]
(1, 10)
[[ 0  1  2  3 -1 -1 -1 -1 -1 -1]]
(1, 10)
[[5 4 6 3 7 2 8 1 9 0]]


WARNING clustering 256 points to 100 centroids: please provide at least 3900 training points
WARNING clustering 256 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 256 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 256 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 256 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 256 points to 256 centroids: please provide at least 9984 training points


In [47]:
import _pickle as pickle

In [48]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp)

# sample usage
save_object(index, 'index.pkl')

In [49]:
with open('index.pkl', 'rb') as inp:
    loaded_index = pickle.load(inp)

In [45]:
loaded_index.is_trained

True

In [23]:
loaded_index.nprobe = 5 
D, I = loaded_index.search(xq, k)     # search
print(I[-5:])

[[5 4 6 3 7 2 8 1 9 0]]
